# Программирование на Python
## Семинар 5. Повторение основ

Имеются данные по сотрудникам некоторой Компании. Они устроены следующим образом. Компания подразделяется на Блоки, Блоки - на Департаменты. Название Департамента необязательно уникально. От отдела кадров по результатам некоторого тестирования вам пришли файлы со статистиками по сотрудникам. Они находятся в папке `data`.

Также существует справочник электронных почт руководителей Департаментов и шаблон письма (папка `sources`).

#### Задача

- изучить файлы от отдела кадров, понять общую структуру;
- на их основании определить среднее значение по каждой из статистик в разрезе Департамента (и проинтерпретировать его - выше или ниже среднего значения по всей компании);
- составить по результатам письмо руководителю каждого из Департаментов;
- автоматически разослать его, используя ваш корпоративный аккаунт.

In [1]:
from pathlib import Path  # для ООП-работы с файловой системой
import openpyxl
from email.mime.text import MIMEText  # для создание email'а по протоколу MIME (для дальнейшей передачи)
from email.mime.multipart import MIMEMultipart
import smtplib  # для SMTP подключения
from configparser import ConfigParser  # для парсинга конфигов
from time import sleep  # для задержки при отправке

In [ ]:
# fio - Справочник (как и email)
# department, block - Справочник / filenames
# value1, value2 - посчитать по каждой таблице
# interp1, interp2 - посчитать среднее по ВСЕМ таблицам и сравнить с ним value1/value2

In [ ]:
# ПЛАН
## функция parse_info (парсим справочник) ++++++
## функция parse_file ++++++
## применяем к всем файлам, сохраняем ++++++
## собираем все значения value1/2 в одном месте -> interp1/2 ++++++
## составить текст сообщения
## сделать email
## отправить email

In [2]:
# constants.py
SOURCE_PATH = Path('sources')
DATA_PATH = Path('data')
MAPPER = {
    'ID': 'id',
    'Показатель 1': 'value1',
    'Показатель 2': 'value2',
}
UNKNOWN_NAME_VALUE = 'UNKNOWN_COL'
INTERP_MAPPER = {
    True: 'значение НИЖЕ среднего по компании',
    False: 'значение не ниже среднего по компании'
}
NDIGITS = 2
NSLEEPSECONDS = 1

In [3]:
# functions.py
def parse_info(path: Path) -> dict:
    wb = openpyxl.load_workbook(path)
    ws = wb.active

    info_dict = {}

    for row in ws.iter_rows():
        if row[0].value != 'Блок':
            pair = {(row[0].value, row[1].value): {'fio': row[2].value, 'email': row[3].value.lower()}}
            info_dict.update(pair)

    wb.close()

    return info_dict

def parse_file(path: Path) -> dict:
    """
    some docstring
    """
    department = path.name.rstrip('.xlsx')
    block = path.parent.name

    wb = openpyxl.load_workbook(path)
    ws = wb.active

    data_dict = {}

    for col in ws.iter_cols():
        values = [cell.value for cell in col if cell.value is not None]
        colname = MAPPER.get(values[0], UNKNOWN_NAME_VALUE)
        colvalues = values[1:]
        data_dict.update({colname: colvalues})

    wb.close()

    return {(block, department): data_dict}

def mean(vector: list) -> float:
    return sum(vector) / len(vector)

In [4]:
# main.py
dict_path = SOURCE_PATH / 'Справочник.xlsx'
config_path = SOURCE_PATH / 'config.ini'
template_path = SOURCE_PATH / 'message_template.html'
filenames = DATA_PATH.glob('Блок */Департамент *.xlsx')

# load template
with open(template_path, 'r') as file:
    template = file.read()

# parse info
info = parse_info(dict_path)

# get params
cp = ConfigParser()
_ = cp.read(config_path)
smtp_configs = cp['smtp.client']

# iterate through files, save data dicts, create value1/2 large vectors
datas = {}

value1 = []
value2 = []

for path in filenames:
    data = parse_file(path)
    datas.update(data)

    table = list(data.values())[0]
    value1.extend(table['value1'])
    value2.extend(table['value2'])

value1_mean = mean(value1)
value2_mean = mean(value2)

# create server object
server = smtplib.SMTP_SSL(smtp_configs['host'], smtp_configs['port'])
login_response = server.login(smtp_configs['login'], smtp_configs['password'])

# iterate through keys (block, department), create message
for key, value in info.items():
    data = datas.get(key)

    if data is None:
        continue
    
    format_dict = {}
    
    # get info values
    block, department = key
    
    format_dict.update(
        {
            'fio': value['fio'],
            'block': block,
            'department': department
        }
    )

    # get datas values
    value1_mean_ = mean(data['value1'])
    value2_mean_ = mean(data['value2'])
    
    format_dict.update(
        {
            'value1': round(value1_mean_, NDIGITS),
            'value2': round(value2_mean_, NDIGITS),
            'interp1': INTERP_MAPPER[value1_mean_ < value1_mean],
            'interp2': INTERP_MAPPER[value2_mean_ < value2_mean]
        }
    )

    # create message text
    message_text = template.format(**format_dict)

    # create email
    email = MIMEMultipart()
    email['From'] = smtp_configs['login']
    email['To'] = value['email']

    subject = f'Отчет по сотрудникам Департамента "{department}" Блока "{block}"'
    email['Subject'] = subject
    
    mimetext = MIMEText(message_text, 'html')
    email.attach(mimetext)

    # send email
    send_response = server.sendmail(email['From'], email['To'], email.as_string())

    if send_response == {}:
        message_to_user = f'Отчет по {block}, {department} отправлен успешно!'
    else:
        message_to_user = f'При отправке отчета по {block}, {department} что-то пошло не так:('

    print(message_to_user)

    # чтобы сервер не принял нас за злоумышленников, сделаем промежуток между запросами в 1 секунду
    sleep(NSLEEPSECONDS)

server.close()

Отчет по Блок 6, Департамент H отправлен успешно!


In [ ]:
# наш код здесь